In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import seaborn as sns

plt.rcdefaults()

# Set options to display all columns and expand column width
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Set options to display a specific number of columns and control column width
pd.set_option('display.max_columns', 10)  # Set the maximum number of columns to display
pd.set_option('display.max_colwidth', 100)  # Set the maximum column width for text data


In [ ]:

plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = (4, 4)
# Set the style of the plot for publication
#sns.set(style="white", context="talk")  # 'talk' context increases font sizes


In [ ]:
pip list

In [ ]:
############multi species analysis

In [ ]:
import pandas as pd
import os


# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]
    
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

combined_pval.fillna(1, inplace=True)

p_value_threshold = 0.05
significant_matrix = combined_pval < p_value_threshold
significant_counts = significant_matrix.sum()

significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Calculate overall normalized percentages
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Add counts of significant samples per species and calculate normalized percentages per species
for species in metadata['species'].unique():
    count_col_name = f'Count_{species}'
    norm_perc_col_name = f'NormalizedPercentage_{species}'

    significant_counts_df[count_col_name] = 0  # Initialize count column

    for index, row in significant_counts_df.iterrows():
        cell_type = row['CellType']
        gene_set = row['GeneSet']
        significant_samples = significant_matrix.loc[significant_matrix[gene_set], gene_set].index
        count_species = sum(sample_to_species.get(sample, "Unknown") == species for sample in significant_samples)
        significant_counts_df.at[index, count_col_name] = count_species

        # Correction: Ensure that normalization is done with the correct denominator
        total_samples_for_cell_type_species = sample_counts_per_cell_type_per_species[species].get(cell_type, 1)
        if total_samples_for_cell_type_species > 0:
            normalized_percentage = (count_species / total_samples_for_cell_type_species) * 100
        else:
            normalized_percentage = 0  # Avoid division by zero
        significant_counts_df.at[index, norm_perc_col_name] = normalized_percentage

# Sort DataFrame by general normalized percentage
sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
print(sorted_df.head())


In [ ]:
# Define a high percentage threshold
high_percentage_threshold = 10  # Adjust this value as needed

# Filter gene sets that are high across all specified species
conserved_features = significant_counts_df[
    (significant_counts_df['NormalizedPercentage_mouse'] > high_percentage_threshold) &
    (significant_counts_df['NormalizedPercentage_rat'] > high_percentage_threshold) &
    (significant_counts_df['NormalizedPercentage_human'] > high_percentage_threshold)
]

# Calculate average normalized percentage across species for the filtered gene sets
conserved_features['AverageNormalizedPercentage'] = conserved_features[
    ['NormalizedPercentage_mouse', 'NormalizedPercentage_rat', 'NormalizedPercentage_human']
].mean(axis=1)

# Sort the filtered DataFrame by average normalized percentage
sorted_conserved_features = conserved_features.sort_values(by='AverageNormalizedPercentage', ascending=False)

sorted_conserved_features.index = sorted_conserved_features["GeneSet"]

# Print the top features based on average normalized percentage
top_conserved_features = sorted_conserved_features.head(50)  # Adjust number as needed
top_conserved_features[:50]

In [ ]:
len(conserved_features)

In [ ]:
len(conserved_features)

In [ ]:

conserved_features.to_csv("/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/conserved_hallmark_features_R1_KEGG.csv")


In [ ]:
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#555555",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Assuming 'sorted_conserved_features' is already available from previous steps

# Select the top 100 features
top_100_features = sorted_conserved_features

# Count the frequency of each CellType in the top 100 features
cell_type_counts = top_100_features['CellType'].value_counts()

# Extract the colors corresponding to the cell types
pie_colors = [cell_colors[cell_type] for cell_type in cell_type_counts.index if cell_type in cell_colors]

# Plotting the pie chart
plt.figure(figsize=(10, 10))  # Adjust the size as needed
cell_type_counts.plot(kind='pie', colors=pie_colors, autopct='')
plt.title('Conserved hallmarks of kidney disease')
plt.ylabel(len(conserved_features))  # Hide the y-label
plt.show()

In [ ]:
conserved_features.head()

In [ ]:
# Assuming conserved_features is already defined and loaded

custom_palette = {
    'NormalizedPercentage_human': 'royalblue',
    'NormalizedPercentage_mouse': '#FF8000',
    'NormalizedPercentage_rat': 'green'
}

# Calculate the average normalized percentage
conserved_features['AverageNormalizedPercentage'] = conserved_features[
    ['NormalizedPercentage_human', 'NormalizedPercentage_mouse', 'NormalizedPercentage_rat']
].mean(axis=1)

# Sort by the average and select the top 5 features
top_5_features = conserved_features.sort_values(by='AverageNormalizedPercentage', ascending=False).head(8)

# Select only the necessary columns for the plot
plot_data = top_5_features[['NormalizedPercentage_human', 'NormalizedPercentage_mouse', 'NormalizedPercentage_rat']]
plot_data['Feature'] = top_5_features['GeneSet']
plot_data = plot_data.melt(id_vars=['Feature'], var_name='Species', value_name='NormalizedPercentage')

# Set the style of the plot for publication
#sns.set(style="white", context="talk")  # 'talk' context increases font sizes

# Adjust figure size and aspect ratio to make bars appear narrower
plt.figure(figsize=(8, 4))  # Increasing the width of the plot

# Create a bar plot
barplot = sns.barplot(x='Feature', y='NormalizedPercentage', hue='Species', data=plot_data, palette=custom_palette)

# Move the y-axis ticks to the right side
plt.gca().yaxis.tick_right()

# Adjust the y-axis label position if necessary
#plt.ylabel('Normalized Percentage', fontsize=16, labelpad=10, rotation=270)


# Add labels and title with increased font size
plt.xlabel('', fontsize=16)
#plt.ylabel('Normalized Percentage', fontsize=16)
plt.title('', fontsize=18)
plt.xticks(rotation=90, fontsize=14)
plt.yticks(rotation=90, fontsize=14)
plt.yticks(fontsize=14)

# Remove the legend if not needed
plt.legend().remove()

# Tight layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Assuming conserved_features is already defined and loaded

custom_palette = {
    'NormalizedPercentage_human': 'royalblue',
    'NormalizedPercentage_mouse': '#FF8000',
    'NormalizedPercentage_rat': 'green'
}

# Calculate the average normalized percentage
conserved_features['AverageNormalizedPercentage'] = conserved_features[
    ['NormalizedPercentage_human', 'NormalizedPercentage_mouse', 'NormalizedPercentage_rat']
].mean(axis=1)

# Sort by the average and select the top 5 features
top_5_features = conserved_features.sort_values(by='AverageNormalizedPercentage', ascending=False).head(8)

# Select only the necessary columns for the plot
plot_data = top_5_features[['NormalizedPercentage_human', 'NormalizedPercentage_mouse', 'NormalizedPercentage_rat']]
plot_data['Feature'] = top_5_features['GeneSet']
plot_data = plot_data.melt(id_vars=['Feature'], var_name='Species', value_name='NormalizedPercentage')

# Set the style of the plot for publication
#sns.set(style="white", context="talk")  # 'talk' context increases font sizes

# Adjust figure size and aspect ratio to make bars appear narrower
plt.figure(figsize=(8, 4))  # Increasing the width of the plot

# Create a bar plot
barplot = sns.barplot(x='Feature', y='NormalizedPercentage', hue='Species', data=plot_data, palette=custom_palette)

# Move the y-axis ticks to the right side
plt.gca().yaxis.tick_right()

# Adjust the y-axis label position if necessary
#plt.ylabel('Normalized Percentage', fontsize=16, labelpad=10, rotation=270)


# Add labels and title with increased font size
plt.xlabel('', fontsize=16)
#plt.ylabel('Normalized Percentage', fontsize=16)
plt.title('', fontsize=18)
plt.xticks(rotation=90, fontsize=14)
plt.yticks(rotation=90, fontsize=14)
plt.yticks(fontsize=14)

# Remove the legend if not needed
plt.legend().remove()

# Tight layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
conserved_features


In [ ]:
cell_type = "PT"

target_gene_set = "hsa04668 TNF signaling pathway - Homo sapiens (human)"

r2_base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"
pval_file_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Padj_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

#group1 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()
group1 = metadata[(metadata["condition_harmonized"] == "CKD") & (metadata["species"] == "human")].orig_ident.unique().tolist()
#group1 = ["A3038", "B3039", "B3035", "J3036", "K3037"]

#group2 = metadata[(metadata["treated"] == "SLGT2i") & (metadata["proj"] == "m_humphreys_DKD")].orig_ident.unique().tolist()
#group2 = metadata[(metadata["disease"] == "diseased") & (metadata["species"] == "mouse")].orig_ident.unique().tolist()
group2 = metadata[(metadata["condition_harmonized"] == "AKI") & (metadata["species"] == "human")].orig_ident.unique().tolist()

samples_to_remove = []
samples_to_remove = ['HK2886'] # Replace with the names of samples you want to remove


# Creating the list with samples to be removed
group1 = [sample for sample in group1 if sample not in samples_to_remove]

group2 = [sample for sample in group2 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.4, 0.6]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.7)  # Set the x-axis range
#plt.xlabel("Groups")
#plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks(group_positions, ["CKD", "AKI"], fontsize=14)  # Set custom x-axis ticks
plt.yticks(fontsize=14)
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(False)
plt.show()

In [ ]:
cell_type = "PT"

target_gene_set = "hsa04150 mTOR signaling pathway - Homo sapiens (human)"

r2_base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"
pval_file_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Padj_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

#group1 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()
group1 = metadata[(metadata["condition_harmonized"] == "CKD") & (metadata["species"] == "human")].orig_ident.unique().tolist()
#group1 = ["A3038", "B3039", "B3035", "J3036", "K3037"]

#group2 = metadata[(metadata["treated"] == "SLGT2i") & (metadata["proj"] == "m_humphreys_DKD")].orig_ident.unique().tolist()
#group2 = metadata[(metadata["disease"] == "diseased") & (metadata["species"] == "mouse")].orig_ident.unique().tolist()
group2 = metadata[(metadata["condition_harmonized"] == "AKI") & (metadata["species"] == "human")].orig_ident.unique().tolist()

samples_to_remove = []
samples_to_remove = ['HK2886'] # Replace with the names of samples you want to remove


# Creating the list with samples to be removed
group1 = [sample for sample in group1 if sample not in samples_to_remove]

group2 = [sample for sample in group2 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.4, 0.6]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.7)  # Set the x-axis range
#plt.xlabel("Groups")
#plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks(group_positions, ["CKD", "AKI"], fontsize=14)  # Set custom x-axis ticks
plt.yticks(fontsize=14)
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(False)
plt.show()

In [ ]:
# Define the path to the R2 file for the specific cell type and gene set
r2_base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"
cell_type = "PT"  # Replace with your desired cell type
target_gene_set = "hsa04066 HIF-1 signaling pathway - Homo sapiens (human)"  # Replace with your desired gene set name

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Padj_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

group0 = [
"K3076",
"K3037",
"I3065",
"J3036",
"N3099",
"M3098",
"B3039",
"N3024",
"Q3067",
"T3097",
"J3079",
"I3078",
"A3038",
"H3069",
"P3066",
"M3096",
"R3077",
"G3068",
"E3075",
"B3035",
"B3025",
"I3095",
"C3028",
"A3026",
"A3029",
"B3027"]

group1 = ["IRI4h1", "IRI4h2", "IRI4h3"]

group2 = ["IRI12h1b1",
"IRI12h1b2",
"IRI12h2",
"IRI12h3"]

group3 = ["IRI2d1b1",
"IRI2d1b2",
"IRI2d2b1",
"IRI2d2b2",
"IRI2d3"]

group4 = ["IRI14d1b1",
"IRI14d1b2",
"IRI14d2",
"IRI14d3"]

group5 = ["IRI6w1b1", "IRI6w1b2", "IRI6w2", "IRI6w3"]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]

# Create a line plot to visualize the mean R2 values for the groups over time
plt.figure(figsize=(10, 6))

# Initialize lists to store mean R2 values and standard deviations for each group
mean_r2_values = []
std_dev_values = []

# Plot each individual sample and color it red if it's significant
for i, group in enumerate([group0, group1, group2, group3, group4, group5]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    std_dev = group_r2_values.std()
    mean_r2_values.append(mean_r2)
    std_dev_values.append(std_dev)

    for sample in group:
        if sample in significant_samples:
            plt.plot(i, group_r2_values.loc[sample], marker='o', markersize=5, color='red')
        else:
            plt.plot(i, group_r2_values.loc[sample], marker='o', markersize=5, color='blue')

# Plot the mean R2 values as a black line with a label
plt.plot(range(6), mean_r2_values, marker='_', color='black', linestyle='--', label='Mean R2')

# Plot standard deviation error bars
#plt.errorbar(range(6), mean_r2_values, yerr=std_dev_values, linestyle='', color='black', elinewidth=2, capsize=5, capthick=2, label='Std Dev')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
#plt.xlabel("Time Points (Groups)")
#plt.ylabel("R2 Value", fontsize=14)
plt.title(f"{target_gene_set} {cell_type}")
plt.xticks(range(6), ["diabetic_control", "4h", "12h", "2d", "14d", "6w"], fontsize=14)
plt.yticks(fontsize=14)
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
#plt.legend()
plt.grid(False)  # Remove the grid
plt.show()

In [ ]:
# Define the path to the R2 file for the specific cell type and gene set
r2_base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"
cell_type = "PT"  # Replace with your desired cell type
target_gene_set = "hsa04668 TNF signaling pathway - Homo sapiens (human)"  # Replace with your desired gene set name

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Padj_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

group0 = [
"K3076",
"K3037",
"I3065",
"J3036",
"N3099",
"M3098",
"B3039",
"N3024",
"Q3067",
"T3097",
"J3079",
"I3078",
"A3038",
"H3069",
"P3066",
"M3096",
"R3077",
"G3068",
"E3075",
"B3035",
"B3025",
"I3095",
"C3028",
"A3026",
"A3029",
"B3027"]

group1 = ["IRI4h1", "IRI4h2", "IRI4h3"]

group2 = ["IRI12h1b1",
"IRI12h1b2",
"IRI12h2",
"IRI12h3"]

group3 = ["IRI2d1b1",
"IRI2d1b2",
"IRI2d2b1",
"IRI2d2b2",
"IRI2d3"]

group4 = ["IRI14d1b1",
"IRI14d1b2",
"IRI14d2",
"IRI14d3"]

group5 = ["IRI6w1b1", "IRI6w1b2", "IRI6w2", "IRI6w3"]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]

# Create a line plot to visualize the mean R2 values for the groups over time
plt.figure(figsize=(10, 6))

# Initialize lists to store mean R2 values and standard deviations for each group
mean_r2_values = []
std_dev_values = []

# Plot each individual sample and color it red if it's significant
for i, group in enumerate([group0, group1, group2, group3, group4, group5]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    std_dev = group_r2_values.std()
    mean_r2_values.append(mean_r2)
    std_dev_values.append(std_dev)

    for sample in group:
        if sample in significant_samples:
            plt.plot(i, group_r2_values.loc[sample], marker='o', markersize=5, color='red')
        else:
            plt.plot(i, group_r2_values.loc[sample], marker='o', markersize=5, color='blue')

# Plot the mean R2 values as a black line with a label
plt.plot(range(6), mean_r2_values, marker='_', color='black', linestyle='--', label='Mean R2')

# Plot standard deviation error bars
#plt.errorbar(range(6), mean_r2_values, yerr=std_dev_values, linestyle='', color='black', elinewidth=2, capsize=5, capthick=2, label='Std Dev')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
#plt.xlabel("Time Points (Groups)")
#plt.ylabel("R2 Value")
plt.title(f"{target_gene_set} {cell_type}")
plt.xticks(range(6), ["diabetic_control", "4h", "12h", "2d", "14d", "6w"], fontsize=14)
plt.yticks(fontsize=14)
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
#plt.legend()
plt.grid(False)  # Remove the grid
plt.show()

In [ ]:

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Padj_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

group0 = metadata[(metadata["species"]=="human") & (metadata["condition_harmonized"]=="CKD")].orig_ident.unique().tolist()

group1 = metadata[(metadata["species"]=="human") & (metadata["condition_harmonized"]=="AKI")].orig_ident.unique().tolist()

samples_to_remove = []
samples_to_remove = ['HK2886']  # Replace with the names of samples you want to remove

# Creating the list with samples to be removed
group0 = [sample for sample in group0 if sample not in samples_to_remove]

group1 = [sample for sample in group1 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.4, 0.6]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group0, group1]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.7)  # Set the x-axis range
#plt.xlabel("Groups")
#plt.ylabel("R2 Value")
plt.title("human")
plt.xticks(group_positions, ["CKD", "AKI"], fontsize=14)  # Set custom x-axis ticks
plt.yticks(fontsize=14)
#plt.legend()
plt.grid(False)
plt.show()

In [ ]:

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Padj_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

group0 = metadata[(metadata["species"]=="rat") & (metadata["condition_harmonized2"]=="H-CKD")].orig_ident.unique().tolist()

group1 = metadata[(metadata["species"]=="rat") & (metadata["condition_harmonized2"]=="DKD")].orig_ident.unique().tolist()

samples_to_remove = []
samples_to_remove = []  # Replace with the names of samples you want to remove


# Creating the list with samples to be removed
group0 = [sample for sample in group0 if sample not in samples_to_remove]

group1 = [sample for sample in group1 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.4, 0.6]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group0, group1]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.7)  # Set the x-axis range
#plt.xlabel("Groups")
#plt.ylabel("R2 Value")
plt.title("rat")
plt.xticks(group_positions, ["H-CKD", "DKD"], fontsize=14)  # Set custom x-axis ticks
plt.yticks(fontsize=14)
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(False)
plt.show()

In [ ]:
# Define the path to the R2 file for the specific cell type and gene set

r2_base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"
cell_type = "PT"  # Replace with your desired cell type
target_gene_set = "hsa04668 TNF signaling pathway - Homo sapiens (human)"  # Replace with your desired gene set name

metadat = metadata

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Padj_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

#group0 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="Control_diseased")].orig_ident.unique().tolist()
group0 = metadata[(metadata['disease']=="diseased") & (metadata['species']=="human")].orig_ident.unique().tolist()

#group1 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="SLGT2i")].orig_ident.unique().tolist()
group1 = metadata[(metadata['disease']=="diseased") & (metadata['species']=="mouse")].orig_ident.unique().tolist()

group2 = metadata[(metadata['disease']=="diseased") & (metadata['species']=="rat")].orig_ident.unique().tolist()


samples_to_remove = []
samples_to_remove = ['31-10006', '32-2', 'HK2558', 'HK2886', 'HK2596', 'IRI2d2b2', 'IRI14d1b2', 'RK7.1', 'RK4.1', 'RK11.2.New', 'RK5.1.Old.MO']  # Replace with the names of samples you want to remove
  # Replace with the names of samples you want to remove

# Creating the list with samples to be removed
group0 = [sample for sample in group0 if sample not in samples_to_remove]

group1 = [sample for sample in group1 if sample not in samples_to_remove]

group2 = [sample for sample in group2 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.4, 0.6, 0.8]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group0, group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.9)  # Set the x-axis range
plt.xlabel(cell_type)
plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks(group_positions, ["Human", "Mouse", "Rat"], rotation = 45)  # Set custom x-axis ticks
plt.yticks(fontsize=14)
#plt.legend()
plt.grid(False)
plt.show()

In [ ]:
# Define the path to the R2 file for the specific cell type and gene set

r2_base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"
cell_type = "PT"  # Replace with your desired cell type
target_gene_set = "hsa04150 mTOR signaling pathway - Homo sapiens (human)"  # Replace with your desired gene set name

metadat = metadata

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Padj_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

#group0 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="Control_diseased")].orig_ident.unique().tolist()
group0 = metadata[(metadata['disease']=="diseased") & (metadata['species']=="human")].orig_ident.unique().tolist()

#group1 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="SLGT2i")].orig_ident.unique().tolist()
group1 = metadata[(metadata['disease']=="diseased") & (metadata['species']=="mouse")].orig_ident.unique().tolist()

group2 = metadata[(metadata['disease']=="diseased") & (metadata['species']=="rat")].orig_ident.unique().tolist()


samples_to_remove = []
samples_to_remove = ['31-10006', '32-2', 'HK2558', 'HK2886', 'HK2596', 'IRI2d2b2', 'IRI14d1b2', 'RK7.1', 'RK4.1', 'RK11.2.New', 'RK5.1.Old.MO']  # Replace with the names of samples you want to remove
  # Replace with the names of samples you want to remove

# Creating the list with samples to be removed
group0 = [sample for sample in group0 if sample not in samples_to_remove]

group1 = [sample for sample in group1 if sample not in samples_to_remove]

group2 = [sample for sample in group2 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.05]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.4, 0.6, 0.8]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group0, group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.9)  # Set the x-axis range
plt.xlabel(cell_type)
plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks(group_positions, ["Human", "Mouse", "Rat"], rotation = 45)  # Set custom x-axis ticks
plt.yticks(fontsize=14)
#plt.legend()
plt.grid(False)
plt.show()

In [ ]:
nan_counts_per_sample

In [ ]:
group2

In [ ]:
nan_counts_per_sample[:50]

In [ ]:
p_value_threshold = 0.05

sample_of_interest = 'GGT_eYFP'

#max_value = 300

ct_plot1 = "Podo"

import re
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label

# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Define your desired order of cell types
desired_order = [
    "PT", 
    "TAL_MD", 
    "DCT_CNT_CD", 
    "IC",    
    "EC",
    "Stromal",      
    "Immune",   
    "DTL_ATL",  
    "PEC",           
    "Podo",    
]

nan_threshold = 100

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]
#cell_types = ["Podo"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]

    ###NEW R1###
    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)
    
    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

# Extract sample-specific features
sample_features = combined_pval.loc[sample_of_interest]

# Filter top features based on p-value threshold
top_20_features = sample_features[sample_features < p_value_threshold]

# Select the top gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

# Count non-NaN pathways per cell type for the sample of interest
non_nan_pathways = combined_pval.loc[sample_of_interest].notna().groupby(lambda x: extract_cell_type(x)).sum()

# Ensure non_nan_pathways aligns with desired_order and fill missing with 0
non_nan_pathways = non_nan_pathways.reindex(desired_order).fillna(0)

# Normalize the significant pathways by the total non-NaN pathways
normalized_values = grouped_data / non_nan_pathways
normalized_values = normalized_values.fillna(0)  # Handle cases where there are no non-NaN pathways

# Initialize ordered values with normalized percentages
ordered_values = [normalized_values.get(ct, 0) * 100 for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
#plt.ylim(0, 40)

# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()

In [ ]:
p_value_threshold = 0.05

sample_of_interest = 'GGT_eYFP'

#max_value = 300

ct_plot1 = "Podo"

import re
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label

# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Define your desired order of cell types
desired_order = [
    "PT", 
    "TAL_MD", 
    "DCT_CNT_CD", 
    "IC",    
    "EC",
    "Stromal",      
    "Immune",   
    "DTL_ATL",  
    "PEC",           
    "Podo",    
]

nan_threshold = 100

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]
#cell_types = ["Podo"]

#this is the mouse only ref - KEGG - filtered
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2/"
pval_folder = "Padj/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]

    ###NEW R1###
    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)
    
    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

# Extract sample-specific features
sample_features = combined_pval.loc[sample_of_interest]

# Filter top features based on p-value threshold
top_20_features = sample_features[sample_features < p_value_threshold]

# Select the top gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

# Count non-NaN pathways per cell type for the sample of interest
non_nan_pathways = combined_pval.loc[sample_of_interest].notna().groupby(lambda x: extract_cell_type(x)).sum()

# Ensure non_nan_pathways aligns with desired_order and fill missing with 0
non_nan_pathways = non_nan_pathways.reindex(desired_order).fillna(0)

# Normalize the significant pathways by the total non-NaN pathways
normalized_values = grouped_data / non_nan_pathways
normalized_values = normalized_values.fillna(0)  # Handle cases where there are no non-NaN pathways

# Initialize ordered values with normalized percentages
ordered_values = [normalized_values.get(ct, 0) * 100 for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
#plt.ylim(0, 40)

# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()

In [ ]:
p_value_threshold = 0.05

sample_of_interest = 'IRI12h1b1'

#max_value = 300

ct_plot1 = "Podo"

import re
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label

# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Define your desired order of cell types
desired_order = [
    "PT", 
    "TAL_MD", 
    "DCT_CNT_CD", 
    "IC",    
    "EC",
    "Stromal",      
    "Immune",   
    "DTL_ATL",  
    "PEC",           
    "Podo",    
]

nan_threshold = 100

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]
#cell_types = ["Podo"]

#this is the mouse only ref - KEGG - filtered
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2/"
pval_folder = "Padj/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]

    ###NEW R1###
    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)
    
    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

# Extract sample-specific features
sample_features = combined_pval.loc[sample_of_interest]

# Filter top features based on p-value threshold
top_20_features = sample_features[sample_features < p_value_threshold]

# Select the top gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

# Count non-NaN pathways per cell type for the sample of interest
non_nan_pathways = combined_pval.loc[sample_of_interest].notna().groupby(lambda x: extract_cell_type(x)).sum()

# Ensure non_nan_pathways aligns with desired_order and fill missing with 0
non_nan_pathways = non_nan_pathways.reindex(desired_order).fillna(0)

# Normalize the significant pathways by the total non-NaN pathways
normalized_values = grouped_data / non_nan_pathways
normalized_values = normalized_values.fillna(0)  # Handle cases where there are no non-NaN pathways

# Initialize ordered values with normalized percentages
ordered_values = [normalized_values.get(ct, 0) * 100 for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
plt.ylim(0, 100)

# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()

In [ ]:
import pandas as pd
import os

# Parameters
p_value_threshold = 0.05
cell_type = "PT"  # Specify the cell type of interest
reference_sample = "IRI12h1b1"  # Define the reference sample
sample_list = ["IRI4h1",
                "IRI4h2",
                "IRI4h3",
    
    
    "IRI12h1b2", "IRI12h2", "IRI12h3",

                "IRI2d1b1",
                "IRI2d1b2",
                "IRI2d2b1",
                "IRI2d2b2",
                "IRI2d3",
               
               "IRI14d1b1",
                "IRI14d1b2",
                "IRI14d2",
                "IRI14d3",

               "IRI6w1b1",
                "IRI6w1b2",
                "IRI6w2",
                "IRI6w3",
              
              ]  # Define the list of samples to compare

# List of all samples of interest (reference + sample_list)
unique_samples = [reference_sample] + sample_list

# Load p-value data for the cell type of interest and filter for unique_samples
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
pval_folder = "Padj/"
pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")

# Load the combined pval data and filter for only the samples of interest (samples as rows, features as columns)
combined_pval = pd.read_csv(pval_path, index_col=0)
combined_pval = combined_pval.loc[unique_samples]

# Initialize a list to store results
results_list = []

# Identify significant gene sets for the reference sample
reference_significant = set(combined_pval.columns[combined_pval.loc[reference_sample] < p_value_threshold])

# Loop through each sample in the sample list and calculate overlap
for sample in sample_list:
    # Identify significant gene sets for the current sample
    sample_significant = set(combined_pval.columns[combined_pval.loc[sample] < p_value_threshold])
    
    # Calculate the union (total significant features across both samples)
    total_significant_union = reference_significant | sample_significant
    num_total_significant = len(total_significant_union)
    
    # Calculate the intersection (overlapping significant features between both samples)
    overlapping_genes = reference_significant & sample_significant
    num_overlap = len(overlapping_genes)
    
    # Calculate the overlap percentage
    overlap_percentage = (num_overlap / num_total_significant) * 100 if num_total_significant > 0 else 0
    
    # Append results as a dictionary to the results list
    results_list.append({
        "Reference-Sample Pair": f"{reference_sample}-{sample}",
        "Significant Features": num_total_significant,
        "Overlap": num_overlap,
        "Overlap Percentage": overlap_percentage, 
        "Sample": sample,
    })

# Convert results list to DataFrame
results = pd.DataFrame(results_list)

# Display the results DataFrame
print(results)





In [ ]:
# Plotting the results
plt.figure(figsize=(8, 6))
plt.plot(results["Sample"], results["Overlap Percentage"], marker='o', linestyle='-')
plt.xlabel("Sample")
plt.ylim(0,100)
plt.ylabel("Overlap Percentage with Reference")
plt.title(f"Overlap Percentage with {reference_sample}")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import os

# Parameters
p_value_threshold = 0.05
cell_type = "PT"  # Specify the cell type of interest
reference_sample = "IRI14d1b1"  # Define the reference sample
sample_list = [
                "IRI14d1b2",
                "IRI14d2",
                "IRI14d3",
    
    "IRI12h1b1", "IRI12h1b2", "IRI12h2", "IRI12h3",

                "IRI2d1b1",
                "IRI2d1b2",
                "IRI2d2b1",
                "IRI2d2b2",
                "IRI2d3",
               
               

               "IRI6w1b1",
                "IRI6w1b2",
                "IRI6w2",
                "IRI6w3",
              
              ]  # Define the list of samples to compare

# List of all samples of interest (reference + sample_list)
unique_samples = [reference_sample] + sample_list

# Load p-value data for the cell type of interest and filter for unique_samples
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
pval_folder = "Padj/"
pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")

# Load the combined pval data and filter for only the samples of interest (samples as rows, features as columns)
combined_pval = pd.read_csv(pval_path, index_col=0)
combined_pval = combined_pval.loc[unique_samples]

# Initialize a list to store results
results_list = []

# Identify significant gene sets for the reference sample
reference_significant = set(combined_pval.columns[combined_pval.loc[reference_sample] < p_value_threshold])

# Loop through each sample in the sample list and calculate overlap
for sample in sample_list:
    # Identify significant gene sets for the current sample
    sample_significant = set(combined_pval.columns[combined_pval.loc[sample] < p_value_threshold])
    
    # Calculate the union (total significant features across both samples)
    total_significant_union = reference_significant | sample_significant
    num_total_significant = len(total_significant_union)
    
    # Calculate the intersection (overlapping significant features between both samples)
    overlapping_genes = reference_significant & sample_significant
    num_overlap = len(overlapping_genes)
    
    # Calculate the overlap percentage
    overlap_percentage = (num_overlap / num_total_significant) * 100 if num_total_significant > 0 else 0
    
    # Append results as a dictionary to the results list
    results_list.append({
        "Reference-Sample Pair": f"{reference_sample}-{sample}",
        "Significant Features": num_total_significant,
        "Overlap": num_overlap,
        "Overlap Percentage": overlap_percentage, 
        "Sample": sample,
    })

# Convert results list to DataFrame
results = pd.DataFrame(results_list)

# Display the results DataFrame
print(results)

# Plotting the results
plt.figure(figsize=(8, 6))
plt.plot(results["Sample"], results["Overlap Percentage"], marker='o', linestyle='-')
plt.xlabel("Sample")
plt.ylim(0,100)
plt.ylabel("Overlap Percentage with Reference")
plt.title(f"Overlap Percentage with {reference_sample}")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
p_value_threshold = 0.05

sample_of_interest = 'PDGFR-eYFP'

#max_value = 300

ct_plot1 = "Podo"

import re
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label

# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Define your desired order of cell types
desired_order = [
    "PT", 
    "TAL_MD", 
    "DCT_CNT_CD", 
    "IC",    
    "EC",
    "Stromal",      
    "Immune",   
    "DTL_ATL",  
    "PEC",           
    "Podo",    
]

nan_threshold = 3000

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]
#cell_types = ["Podo"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_GO_0/"
pval_folder = "Padj_GO_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]

    ###NEW R1###
    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)
    
    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

# Extract sample-specific features
sample_features = combined_pval.loc[sample_of_interest]

# Filter top features based on p-value threshold
top_20_features = sample_features[sample_features < p_value_threshold]

# Select the top gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

# Count non-NaN pathways per cell type for the sample of interest
non_nan_pathways = combined_pval.loc[sample_of_interest].notna().groupby(lambda x: extract_cell_type(x)).sum()

# Ensure non_nan_pathways aligns with desired_order and fill missing with 0
non_nan_pathways = non_nan_pathways.reindex(desired_order).fillna(0)

# Normalize the significant pathways by the total non-NaN pathways
normalized_values = grouped_data / non_nan_pathways
normalized_values = normalized_values.fillna(0)  # Handle cases where there are no non-NaN pathways

# Initialize ordered values with normalized percentages
ordered_values = [normalized_values.get(ct, 0) * 100 for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
#plt.ylim(0, 40)

# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()

In [ ]:
p_value_threshold = 0.05

sample_of_interest = 'GGT_eYFP'

#max_value = 300

ct_plot1 = "Podo"

import re
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label

# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Define your desired order of cell types
desired_order = [
    "PT", 
    "TAL_MD", 
    "DCT_CNT_CD", 
    "IC",    
    "EC",
    "Stromal",      
    "Immune",   
    "DTL_ATL",  
    "PEC",           
    "Podo",    
]

nan_threshold = 3000

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]
#cell_types = ["Podo"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_GO_0/"
pval_folder = "Padj_GO_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]

    ###NEW R1###
    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)
    
    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

# Extract sample-specific features
sample_features = combined_pval.loc[sample_of_interest]

# Filter top features based on p-value threshold
top_20_features = sample_features[sample_features < p_value_threshold]

# Select the top gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

# Count non-NaN pathways per cell type for the sample of interest
non_nan_pathways = combined_pval.loc[sample_of_interest].notna().groupby(lambda x: extract_cell_type(x)).sum()

# Ensure non_nan_pathways aligns with desired_order and fill missing with 0
non_nan_pathways = non_nan_pathways.reindex(desired_order).fillna(0)

# Normalize the significant pathways by the total non-NaN pathways
normalized_values = grouped_data / non_nan_pathways
normalized_values = normalized_values.fillna(0)  # Handle cases where there are no non-NaN pathways

# Initialize ordered values with normalized percentages
ordered_values = [normalized_values.get(ct, 0) * 100 for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
#plt.ylim(0, 40)

# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()

In [ ]:
p_value_threshold = 0.05

sample_of_interest = "PDGFR-eYFP"

#max_value = 300

ct_plot1 = "PT"

#specify how many nan we tolerate (critical if applying gene filtering during spectra analysis)
nan_threshold = 1000

import pandas as pd
import os

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]

#cell_types = ["PEC", "Immune"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]
    
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1



import re

import re

def auto_split_label(label, max_length=40):
    # Remove any cell-type prefix before "hsa" (such as "PT_", "DCT_CNT_CD_", "DTL_ATL_", etc.)
    label = re.sub(r'^[A-Za-z0-9_]+_hsa\d+\s+', '', label)  # Remove cell-type prefixes like "PT_", "DCT_CNT_CD_", etc.

    # Remove the "hsaxxxxx" prefix and " - Homo sapiens (human)" suffix
    label = re.sub(r'^hsa\d+\s+', '', label)  # Remove "hsaxxxxx" prefix if it's there
    label = re.sub(r'\s+-\s+Homo\s+sapiens\s+\(human\)$', '', label)  # Remove " - Homo sapiens (human)" suffix

    # Split the remaining pathway name into words and process the splitting
    words = label.split()
    
    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label


# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Sample data - replace with your actual data

top_20_features = combined_pval.loc[sample_of_interest].nsmallest(10)
neg_log_pvals = -np.log10(top_20_features)
r2_values = combined_r2.loc[sample_of_interest][top_20_features.index]

# Create a reversed red colormap for R2 values
cmap = plt.cm.Reds_r  # '_r' suffix to reverse the colormap

norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalization fixed from 0 to 1

# Assuming 'significant_counts_df' is your DataFrame and it has a 'GeneSet' and 'NormalizedPercentage' column
scale_factor = 4
base_size = 100
#base_size = 20

# Map features to their normalized percentage
feature_to_percentage = dict(zip(significant_counts_df['GeneSet'], significant_counts_df['NormalizedPercentage']))

# Create the dot plot
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(r2_values[feature]))
    
    # Get the normalized percentage for the dot size
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(top_20_features.index, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title(f'Top Features for Sample {sample_of_interest}', fontsize=14)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left')
ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()









# Create the dot plot
fig, ax = plt.subplots(figsize=(10, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    
    # Draw the bar for significance
    ax.plot([0, neg_log_pval], [i, i], color=bar_color, linewidth=3)


# Process each feature name through auto_split_label and set as y-tick labels
processed_labels = [auto_split_label(feat) for feat in top_20_features.index]
ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(processed_labels, fontsize=12)

#ax.set_yticklabels(top_20_features.index, fontsize=20)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

plt.tight_layout()
plt.show()









# Define your desired order of cell types
desired_order = [
     
"PT", 
"TAL_MD", 
"DCT_CNT_CD", 
"IC",    
"EC",
"Stromal",      
"Immune",   
"DTL_ATL",  
"PEC",           
"Podo",    

    
]  # Replace with actual cell types

#top_20_features = combined_pval.loc[sample_of_interest].nsmallest(100)

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
#plt.ylim(0, 1200)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)


#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


# Desired cell type for visualization
desired_cell_type = ct_plot1

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(20)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()

In [ ]:
p_value_threshold = 0.05

sample_of_interest = "PDGFR-eYFP"

#max_value = 300

ct_plot1 = "PT"

#specify how many nan we tolerate (critical if applying gene filtering during spectra analysis)
nan_threshold = 1000

import pandas as pd
import os

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]

#cell_types = ["PEC", "Immune"]

#this is the mouse only ref
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2/"
pval_folder = "Padj/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]
    
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1



import re

import re

def auto_split_label(label, max_length=40):
    # Remove any cell-type prefix before "hsa" (such as "PT_", "DCT_CNT_CD_", "DTL_ATL_", etc.)
    label = re.sub(r'^[A-Za-z0-9_]+_mmu\d+\s+', '', label)  # Remove cell-type prefixes like "PT_", "DCT_CNT_CD_", etc.

    # Remove the "hsaxxxxx" prefix and " - Homo sapiens (human)" suffix
    label = re.sub(r'^hsa\d+\s+', '', label)  # Remove "hsaxxxxx" prefix if it's there
    label = re.sub(r'\s+-\s+Mus\s+musculus\s+\(house mouse\)$', '', label)  # Remove " - Homo sapiens (human)" suffix

    # Split the remaining pathway name into words and process the splitting
    words = label.split()
    
    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label


# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Sample data - replace with your actual data

top_20_features = combined_pval.loc[sample_of_interest].nsmallest(10)
neg_log_pvals = -np.log10(top_20_features)
r2_values = combined_r2.loc[sample_of_interest][top_20_features.index]

# Create a reversed red colormap for R2 values
cmap = plt.cm.Reds_r  # '_r' suffix to reverse the colormap

norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalization fixed from 0 to 1

# Assuming 'significant_counts_df' is your DataFrame and it has a 'GeneSet' and 'NormalizedPercentage' column
scale_factor = 4
base_size = 100
#base_size = 20

# Map features to their normalized percentage
feature_to_percentage = dict(zip(significant_counts_df['GeneSet'], significant_counts_df['NormalizedPercentage']))

# Create the dot plot
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(r2_values[feature]))
    
    # Get the normalized percentage for the dot size
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(top_20_features.index, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title(f'Top Features for Sample {sample_of_interest}', fontsize=14)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left')
ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()









# Create the dot plot
fig, ax = plt.subplots(figsize=(10, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    
    # Draw the bar for significance
    ax.plot([0, neg_log_pval], [i, i], color=bar_color, linewidth=3)


# Process each feature name through auto_split_label and set as y-tick labels
processed_labels = [auto_split_label(feat) for feat in top_20_features.index]
ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(processed_labels, fontsize=12)

#ax.set_yticklabels(top_20_features.index, fontsize=20)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

plt.tight_layout()
plt.show()









# Define your desired order of cell types
desired_order = [
     
"PT", 
"TAL_MD", 
"DCT_CNT_CD", 
"IC",    
"EC",
"Stromal",      
"Immune",   
"DTL_ATL",  
"PEC",           
"Podo",    

    
]  # Replace with actual cell types

#top_20_features = combined_pval.loc[sample_of_interest].nsmallest(100)

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
#plt.ylim(0, 1200)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)


#########THIS WAS FIXED############
# List to store cell types with all NaN values
missing_samples = []

# Loop through each cell type
for celltype in desired_order:
    # Subset 'allfeatures' to only include features starting with the current cell type
    celltype_features = sample_features[sample_features.index.str.startswith(celltype)]
    
    # Check if the number of NaN values is above the threshold
    if celltype_features.isna().sum() > nan_threshold:
        # Add the celltype to the missing_samples list
        missing_samples.append(celltype)

for idx, label in enumerate(ax.get_xticklabels()):
    # Check if the cell type is in the missing_samples list
    if desired_order[idx] in missing_samples:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


# Desired cell type for visualization
desired_cell_type = ct_plot1

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(20)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()

In [ ]:
# Sample groups
group1 = ["IRI4h1", "IRI4h2", "IRI4h3"]

group2 = ["IRI12h1b1",
"IRI12h1b2",
"IRI12h2",
"IRI12h3"]

group3 = ["IRI2d1b1",
"IRI2d1b2",
"IRI2d2b1",
"IRI2d2b2",
"IRI2d3"]

group4 = ["IRI14d1b1",
"IRI14d1b2",
"IRI14d2",
"IRI14d3"]

group5 = ["IRI6w1b1", "IRI6w1b2", "IRI6w2", "IRI6w3"]

color_list = ['skyblue', 'lightgreen', 'salmon', "lightblue", "yellow"]

groups = {'4h': group1, '12h': group2, '2d': group3, '14d': group4, '6W': group5}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.01

    # Prepare DataFrame to hold counts
    counts_df = pd.DataFrame(columns=['Group', 'Count'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count gene sets for the specific cell type
            count = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Append count to the DataFrame only if count is greater than zero
            if count > 0:
                new_row = pd.DataFrame({'Group': [group_name], 'Count': [count]})
                counts_df = pd.concat([counts_df, new_row], ignore_index=True)

    # Plotting the counts using box plot
    plt.figure(figsize=(10, 3))
    sns.boxplot(data=counts_df, x='Group', y='Count', palette=color_list)
    sns.swarmplot(data=counts_df, x='Group', y='Count', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Number of Significant Gene Sets')
    plt.ylim(0)
    plt.xlabel('Group')
    plt.show()

In [ ]:
p_value_threshold = 0.05

sample_of_interest = 'RK9.1.Old.MO'

#max_value = 300

ct_plot1 = "Podo"

import re
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label

# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Define your desired order of cell types
desired_order = [
    "PT", 
    "TAL_MD", 
    "DCT_CNT_CD", 
    "IC",    
    "EC",
    "Stromal",      
    "Immune",   
    "DTL_ATL",  
    "PEC",           
    "Podo",    
]

nan_threshold = 100

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]
#cell_types = ["Podo"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]

    ###NEW R1###
    # Count the number of NaN values per sample (row) in the combined_pval DataFrame
    nan_counts_per_sample = pval_df.isna().sum(axis=1)
    
    # Exclude samples with more than the defined threshold of NaNs
    samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
    r2_df = r2_df.drop(samples_to_exclude, axis=0)
    pval_df = pval_df.drop(samples_to_exclude, axis=0)

    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')



# Sample groups
group1 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()
group2 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_doca")].orig_ident.unique().tolist()

color_list = ['green', 'lightgreen']

groups = {'ZSF': group1, 'DOCA': group2}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.05

    # Prepare DataFrame to hold percentages
    percentage_df = pd.DataFrame(columns=['Group', 'Percentage'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]

            # Count total non-NaN gene sets for the specific cell type
            total_non_nan = sample_features.notna().index.map(extract_cell_type).to_series().apply(lambda x: x == cell_type_of_interest).sum()

            if total_non_nan == 0:
                continue  # Skip if no non-NaN pathways for this cell type

            # Filter for significant features (below the p-value threshold)
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count significant gene sets for the specific cell type
            count_significant = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Calculate the percentage of significant gene sets relative to the total non-NaN gene sets
            percentage = (count_significant / total_non_nan) * 100


            new_row = pd.DataFrame({'Group': [group_name], 'Percentage': [percentage]})
            percentage_df = pd.concat([percentage_df, new_row], ignore_index=True)

    # Plotting the percentages using a box plot
    plt.figure(figsize=(5, 3))
    sns.boxplot(data=percentage_df, x='Group', y='Percentage', palette=color_list)
    sns.swarmplot(data=percentage_df, x='Group', y='Percentage', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Percentage of Significant Gene Sets')
    #plt.ylim(0, 100)
    plt.xlabel('Group')
    plt.show()

In [ ]:
counts_df

In [ ]:
import pandas as pd
import os

nan_threshold = 100

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]

#cell_types = ["PEC", "Immune"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]
    
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

# Sample groups
group1 = ["IRI4h1", "IRI4h2", "IRI4h3"]

group2 = ["IRI12h1b1",
"IRI12h1b2",
"IRI12h2",
"IRI12h3"]

group3 = ["IRI2d1b1",
"IRI2d1b2",
"IRI2d2b1",
"IRI2d2b2",
"IRI2d3"]

group4 = ["IRI14d1b1",
"IRI14d1b2",
"IRI14d2",
"IRI14d3"]

group5 = ["IRI6w1b1", "IRI6w1b2", "IRI6w2", "IRI6w3"]

color_list = ['skyblue', 'lightgreen', 'salmon', "lightblue", "yellow"]

groups = {'4h': group1, '12h': group2, '2d': group3, '14d': group4, '6W': group5}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.05

    # Prepare DataFrame to hold percentages
    percentage_df = pd.DataFrame(columns=['Group', 'Percentage'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]

            # Count total non-NaN gene sets for the specific cell type
            total_non_nan = sample_features.notna().index.map(extract_cell_type).to_series().apply(lambda x: x == cell_type_of_interest).sum()

            if total_non_nan == 0:
                continue  # Skip if no non-NaN pathways for this cell type

            # Filter for significant features (below the p-value threshold)
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count significant gene sets for the specific cell type
            count_significant = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Calculate the percentage of significant gene sets relative to the total non-NaN gene sets
            percentage = (count_significant / total_non_nan) * 100


            new_row = pd.DataFrame({'Group': [group_name], 'Percentage': [percentage]})
            percentage_df = pd.concat([percentage_df, new_row], ignore_index=True) 
            
    # Plotting the counts using box plot
    plt.figure(figsize=(10, 3))
    sns.boxplot(data=percentage_df, x='Group', y='Percentage', palette=color_list)
    sns.swarmplot(data=percentage_df, x='Group', y='Percentage', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Percentage of Significant Gene Sets')
    plt.ylim(0)
    plt.xlabel('Group')
    plt.show()

In [ ]:
import pandas as pd
import os

nan_threshold = 100

# Define cell types and file paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]

#cell_types = ["PEC", "Immune"]

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]
    
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

# Sample groups
group1 = ["N3024", "A3026", "A3029", "B3025", "B3027", "C3028"]

group2 = ["A3038", "B3039", "B3035", "J3036", "K3037"]




color_list = ['yellow', 'orange']

groups = {'db/db': group1, 'db/db+AAV': group2}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.05

    # Prepare DataFrame to hold percentages
    percentage_df = pd.DataFrame(columns=['Group', 'Percentage'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]

            # Count total non-NaN gene sets for the specific cell type
            total_non_nan = sample_features.notna().index.map(extract_cell_type).to_series().apply(lambda x: x == cell_type_of_interest).sum()

            if total_non_nan == 0:
                continue  # Skip if no non-NaN pathways for this cell type

            # Filter for significant features (below the p-value threshold)
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count significant gene sets for the specific cell type
            count_significant = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Calculate the percentage of significant gene sets relative to the total non-NaN gene sets
            percentage = (count_significant / total_non_nan) * 100


            new_row = pd.DataFrame({'Group': [group_name], 'Percentage': [percentage]})
            percentage_df = pd.concat([percentage_df, new_row], ignore_index=True) 

    # Plotting the counts using box plot
    plt.figure(figsize=(5, 3))
    sns.boxplot(data=percentage_df, x='Group', y='Percentage', palette=color_list)
    sns.swarmplot(data=percentage_df, x='Group', y='Percentage', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Percentage of Significant Gene Sets')
    plt.ylim(0)
    plt.xlabel('Group')
    plt.show()

In [ ]:
# Replace with your actual file paths and cell types
cell_types = ["PT"] # etc.
# Paths as specified in your first script
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/"
r2_folder = "output_SISKA_crossspecies_healthy_GSEA/"
pval_folder = "output_SISKA_crossspecies_healthy_GSEA/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"NES_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    pval_df = pd.read_csv(pval_path, index_col=0)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)


# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = combined_r2

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

sorted_df.to_csv("/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy_GSEA/GSEA_FA_PT_Hallmarks_siska.csv")

In [ ]:
# Replace with your actual file paths and cell types
cell_types = ["PT"] # etc.
# Paths as specified in your first script
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2_GSEA/"
pval_folder = "Pval_GSEA/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"NES_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    pval_df = pd.read_csv(pval_path, index_col=0)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)


# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = combined_r2

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

sorted_df.to_csv("/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/GSEA_FA_PT_Hallmarks_mouseref.csv")

In [ ]:
# Replace with your actual file paths and cell types
cell_types = ["PT"] # etc.
# Paths as specified in your first script
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    pval_df = pd.read_csv(pval_path, index_col=0)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)


# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = combined_r2

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)

sorted_df.to_csv("/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/CellSpectra_FA_PT_Hallmarks_siska.csv")

In [ ]:
# Replace with your actual file paths and cell types
cell_types = ["PT"] # etc.
# Paths as specified in your first script
base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2/"
pval_folder = "Padj/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadat = pd.read_csv(metadat_path)
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    pval_df = pd.read_csv(pval_path, index_col=0)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)


# Scale the combined_r2 data
# Note: The fit_transform method expects a 2D array, so we use .values to convert the DataFrame
min_max_scaled_r2_values = combined_r2

# Create a new DataFrame from the min-max scaled data, preserving the index and column names

combined_r2 = pd.DataFrame(min_max_scaled_r2_values, index=combined_r2.index, columns=combined_r2.columns)

# Set p-value threshold
p_value_threshold = 0.05

# Identify significant gene sets
significant_matrix = combined_pval < p_value_threshold

# Count significant occurrences
significant_counts = significant_matrix.sum()

# Create DataFrame for counts and cell types
significant_counts_df = pd.DataFrame({'GeneSet': significant_counts.index, 'Count': significant_counts.values})

# Apply the function to extract the full cell type name
significant_counts_df['CellType'] = significant_counts_df['GeneSet'].apply(extract_cell_type)

# Normalize by sample number per cell type
significant_counts_df['NormalizedPercentage'] = significant_counts_df.apply(
    lambda row: (row['Count'] / sample_counts_per_cell_type.get(row['CellType'], 1)) * 100, axis=1
)

# Display final DataFrame
print(significant_counts_df.head())  # You can modify this to display as needed

sorted_df = significant_counts_df.sort_values(by='NormalizedPercentage', ascending=False)
display(sorted_df)
sorted_df.to_csv("/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/CellSpectra_FA_PT_Hallmarks_mouseref.csv")

In [ ]:
combined_pval

In [ ]:
top_common_features = [
   "PT_hsa04150 mTOR signaling pathway - Homo sapiens (human)",
    "PT_hsa04668 TNF signaling pathway - Homo sapiens (human)", 
    #"PT_hsa04066 HIF-1 signaling pathway - Homo sapiens (human)",
    #"PT_hsa04350 TGF-beta signaling pathway - Homo sapiens (human)"

]

#top_common_features = list(set(top_common_features))

cell_types = ["PT"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 4000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/"
r2_folder = "output_SISKA_crossspecies_healthy_GSEA/"
pval_folder = "output_SISKA_crossspecies_healthy_GSEA/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"NES_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Set p-value threshold
p_value_threshold = 0.05


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 0, combined_r2[col])

# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]

# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

# Function to clean pathway labels
def clean_pathway_label(label):
    # Remove the cell-type prefix (e.g., "PT_") and "hsaxxxxx" and the suffix " - Homo sapiens (human)"
    label = re.sub(r'^[A-Za-z0-9_]+_hsa\d+\s+', '', label)  # Remove "PT_", "DCT_CNT_CD_", etc.
    label = re.sub(r'\s+-\s+Homo\s+sapiens\s+\(human\)$', '', label)  # Remove " - Homo sapiens (human)" suffix
    return label

# Apply the cleaning function to top_common_features
specific_functions = [clean_pathway_label(feature) for feature in top_common_features]

# Remove function names from column labels (clean any existing labels)
plotting_matrix.columns = specific_functions

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="RdBu_r", figsize=(24, 6), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.0, 0.0, 0.0, 0.0), vmin=-4, vmax=4)

plt.show()

In [ ]:
combined_r2

In [ ]:
top_common_features = [
   "PT_hsa04150 mTOR signaling pathway - Homo sapiens (human)",
    "PT_hsa04668 TNF signaling pathway - Homo sapiens (human)", 

]

#top_common_features = list(set(top_common_features))

cell_types = ["PT"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 3000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)
not_counts_per_sample = combined_pval.notna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Set p-value threshold
p_value_threshold = 0.05


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 1, combined_r2[col])

# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]



# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

# Function to clean pathway labels
def clean_pathway_label(label):
    # Remove the cell-type prefix (e.g., "PT_") and "hsaxxxxx" and the suffix " - Homo sapiens (human)"
    label = re.sub(r'^[A-Za-z0-9_]+_hsa\d+\s+', '', label)  # Remove "PT_", "DCT_CNT_CD_", etc.
    label = re.sub(r'\s+-\s+Homo\s+sapiens\s+\(human\)$', '', label)  # Remove " - Homo sapiens (human)" suffix
    return label

# Apply the cleaning function to top_common_features
specific_functions = [clean_pathway_label(feature) for feature in top_common_features]

# Remove function names from column labels (clean any existing labels)
plotting_matrix.columns = specific_functions

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="rocket", figsize=(24, 6), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.00, 0.00, 0.00, 0.00), vmin=0, vmax=1)

plt.show()

In [ ]:
top_common_features = [
   "PT_mmu04150 mTOR signaling pathway - Mus musculus (house mouse)",
    "PT_mmu04668 TNF signaling pathway - Mus musculus (house mouse)", 

]

#top_common_features = list(set(top_common_features))

cell_types = ["PT"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 3000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2/"
pval_folder = "Padj/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)
not_counts_per_sample = combined_pval.notna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Set p-value threshold
p_value_threshold = 0.05


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 1, combined_r2[col])

# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]



# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

# Function to clean pathway labels
def clean_pathway_label(label):
    # Remove the cell-type prefix (e.g., "PT_") and "hsaxxxxx" and the suffix " - Homo sapiens (human)"
    label = re.sub(r'^[A-Za-z0-9_]+_mmu\d+\s+', '', label)  # Remove "PT_", "DCT_CNT_CD_", etc.
    label = re.sub(r'\s+-\s+Mus\s+musculus\s+\(house mouse\)$', '', label)  # Remove " - Homo sapiens (human)" suffix
    return label

# Apply the cleaning function to top_common_features
specific_functions = [clean_pathway_label(feature) for feature in top_common_features]

# Remove function names from column labels (clean any existing labels)
plotting_matrix.columns = specific_functions

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="rocket", figsize=(24, 6), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.00, 0.00, 0.00, 0.00), vmin=0, vmax=1)

plt.show()

In [ ]:
top_common_features = [
   "PT_mmu04150 mTOR signaling pathway - Mus musculus (house mouse)",
    "PT_mmu04668 TNF signaling pathway - Mus musculus (house mouse)", 

]

#top_common_features = list(set(top_common_features))

cell_types = ["PT"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 3000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2/"
pval_folder = "Padj/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)
not_counts_per_sample = combined_pval.notna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Set p-value threshold
p_value_threshold = 0.05


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 1, combined_r2[col])

# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]



# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

# Function to clean pathway labels
def clean_pathway_label(label):
    # Remove the cell-type prefix (e.g., "PT_") and "hsaxxxxx" and the suffix " - Homo sapiens (human)"
    label = re.sub(r'^[A-Za-z0-9_]+_mmu\d+\s+', '', label)  # Remove "PT_", "DCT_CNT_CD_", etc.
    label = re.sub(r'\s+-\s+Mus\s+musculus\s+\(house mouse\)$', '', label)  # Remove " - Homo sapiens (human)" suffix
    return label

# Apply the cleaning function to top_common_features
specific_functions = [clean_pathway_label(feature) for feature in top_common_features]

# Remove function names from column labels (clean any existing labels)
plotting_matrix.columns = specific_functions

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="rocket", figsize=(24, 6), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.00, 0.00, 0.00, 0.00), vmin=0, vmax=1)

plt.show()

In [ ]:
import re

top_common_features = [
   "PT_mmu04150 mTOR signaling pathway - Mus musculus (house mouse)",
    "PT_mmu04668 TNF signaling pathway - Mus musculus (house mouse)", 

]

#top_common_features = list(set(top_common_features))

cell_types = ["PT"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 3000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2/"
pval_folder = "Padj/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold


# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)
not_counts_per_sample = combined_pval.notna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Set p-value threshold
p_value_threshold = 0.05


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 1, combined_r2[col])

# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]



# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

# Function to clean pathway labels
def clean_pathway_label(label):
    # Remove the cell-type prefix (e.g., "PT_") and "hsaxxxxx" and the suffix " - Homo sapiens (human)"
    label = re.sub(r'^[A-Za-z0-9_]+_mmu\d+\s+', '', label)  # Remove "PT_", "DCT_CNT_CD_", etc.
    label = re.sub(r'\s+-\s+Mus\s+musculus\s+\(house mouse\)$', '', label)  # Remove " - Homo sapiens (human)" suffix
    return label

# Apply the cleaning function to top_common_features
specific_functions = [clean_pathway_label(feature) for feature in top_common_features]

# Remove function names from column labels (clean any existing labels)
plotting_matrix.columns = specific_functions

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="rocket", figsize=(24, 6), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.00, 0.00, 0.00, 0.00), vmin=0, vmax=1)

plt.show()

In [ ]:
top_common_features = [
   "PT_mmu04150 mTOR signaling pathway - Mus musculus (house mouse)",

]

#top_common_features = list(set(top_common_features))

cell_types = ["PT"]

# Define the threshold for the maximum number of NaNs allowed
nan_threshold = 3000  # Example threshold, you can adjust this value

base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/scSpectra_Revision/scSpectra_Mousekidney/output"
r2_folder = "R2_GSEA/"
pval_folder = "Pval_GSEA/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"  # or some default value

# Read metadata
metadata = pd.read_csv(metadat_path)
metadat = metadata

# Step 2: Select columns of interest
#columns_of_interest = ['Donor_id', 'Sex', 'Age', "Age_group"]
#metadat = metadat[columns_of_interest]

#metadata = metadat.drop_duplicates(subset=['Donor_id'])

#unique_samples = metadata[(metadata['Age_group'] != "A")]['Donor_id'].unique()
#unique_samples = metadat[(metadat['Disease_level2'] != "Control") & (metadat['Project/Dataset'] == "cal_CKD")]['Sample'].unique()
#unique_samples = ["LYZ2_eYFP", "FA3", "GGT_eYFP", "PDGFR-eYFP"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['uicc_stage'] != "I") & (metadata['platform'] == "10x")]['patient'].unique()
unique_samples = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]
#unique_samples = metadata[(metadata['tissue: tumor_primary'] == 1.0) & (metadata['platform'] == "10x") & (metadata['driver_genes'] == "EGRF")]['patient'].unique()
#yes_list = ["EGRF"]
#unique_samples = metadata[(metadata['platform'] == "10x") & (metadata['driver_genes'].isin(yes_list))]["patient"].unique()  # Samples above the threshold
import re

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"NES_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0, skipinitialspace=True)
    pval_df = pd.read_csv(pval_path, index_col=0, skipinitialspace=True)
    
    # Modify column names
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    # Merge data
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    # Count samples per cell type
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]




# Count the number of NaN values per sample (row) in the combined_pval DataFrame
nan_counts_per_sample = combined_pval.isna().sum(axis=1)
not_counts_per_sample = combined_pval.notna().sum(axis=1)

# Exclude samples with more than the defined threshold of NaNs
samples_to_exclude = nan_counts_per_sample[nan_counts_per_sample > nan_threshold].index
combined_r2 = combined_r2.drop(samples_to_exclude, axis=0)
combined_pval = combined_pval.drop(samples_to_exclude, axis=0)

# Ensure there are no all-NaN columns before scaling
combined_r2 = combined_r2.dropna(axis=1, how='all')

# Ensure there are no all-NaN columns before scaling
combined_pval = combined_pval.dropna(axis=1, how='all')

# Handle NA values in p-values
combined_pval.fillna(1, inplace=True)

# Set p-value threshold
p_value_threshold = 1


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Define your specific functions and cell type
specific_functions = top_common_features  # replace with actual function names

# Remove samples that are all NaN in combined_r2
non_nan_samples = combined_r2.dropna(how='all').index
combined_r2 = combined_r2.loc[non_nan_samples]
combined_pval = combined_pval.loc[non_nan_samples]

# Set non-significant R2 values to 1

for col in combined_r2.columns:
    combined_r2[col] = np.where((combined_pval[col] > p_value_threshold), 0, combined_r2[col])

# Filter for the specific functions across the single cell type
plotting_matrix = combined_r2[specific_functions]

# Remove function names from column labels
plotting_matrix.columns = specific_functions

# Define your custom row order as a list of row indices or labels
custom_order = ["LYZ2_eYFP", "PDGFR-eYFP", "GGT_eYFP", "FA3"]



# Reorder the rows
plotting_matrix = plotting_matrix.loc[custom_order]

# Function to clean pathway labels
def clean_pathway_label(label):
    # Remove the cell-type prefix (e.g., "PT_") and "hsaxxxxx" and the suffix " - Homo sapiens (human)"
    label = re.sub(r'^[A-Za-z0-9_]+_mmu\d+\s+', '', label)  # Remove "PT_", "DCT_CNT_CD_", etc.
    label = re.sub(r'\s+-\s+Mus\s+musculus\s+\(house mouse\)$', '', label)  # Remove " - Homo sapiens (human)" suffix
    return label

# Apply the cleaning function to top_common_features
specific_functions = [clean_pathway_label(feature) for feature in top_common_features]

# Remove function names from column labels (clean any existing labels)
plotting_matrix.columns = specific_functions

plotting_matrix = plotting_matrix.T

sns.set(font_scale=2)

# Plotting without dendrogram and custom NaN color
plt.figure(figsize=(48, 48))
sns.clustermap(plotting_matrix, cmap="RdBu_r", figsize=(24, 6), 
               linewidths=.1, linecolor='black', row_cluster=False, col_cluster=False, 
               cbar_pos=(0.0, 0.1, 0.1, 0.3), vmin=-2, vmax=2)

plt.show()

In [ ]:
sample_of_interest = 'RK16_4'

max_value = 1000

ct_plot1 = "Podo"

import re

import re

def auto_split_label(label, max_length=40):

    extracted_name = re.search(r'.*?_GO\.\d{7}\.(.*?)\.[A-Z]{2}$', label)
    if extracted_name:
        # Replace dots with spaces and split into words
        words = extracted_name.group(1).replace('.', ' ').split()
    else:
        # If no match, return an empty string or some placeholder
        return ""

    # Start with the first word and check length as adding more words
    split_label = words[0]
    current_length = len(words[0])

    # Add each word to the line until the max_length is reached, then start a new line
    for word in words[1:]:
        if current_length + len(word) + 1 <= max_length:
            split_label += ' ' + word
            current_length += len(word) + 1
        else:
            split_label += '\n' + word
            current_length = len(word)

    return split_label


base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy"
r2_folder = "R2_KEGG_0/"
pval_folder = "Padj_KEGG_0/"
metadata_path = "/home/isilon/users/o_kloetzer/Atlas/scSPECTRA/multispecies/multispecies_metadata_complete.csv"
metadat_path = metadata_path

def extract_cell_type(gene_set):
    for cell_type in cell_types:
        if gene_set.startswith(cell_type):
            return cell_type
    return "Unknown"

# Read metadata
metadata = pd.read_csv(metadata_path)
unique_samples = metadata[metadata['disease'] == "diseased"]['orig_ident'].unique()
sample_to_species = metadata.set_index('orig_ident')['species'].to_dict()

# Initialize combined dataframes for R2 and p-values
combined_r2 = pd.DataFrame(index=unique_samples)
combined_pval = pd.DataFrame(index=unique_samples)
sample_counts_per_cell_type = {}
sample_counts_per_cell_type_per_species = {species: {ct: 0 for ct in cell_types} for species in metadata['species'].unique()}

# Process each cell type
for cell_type in cell_types:
    r2_path = os.path.join(base_path, r2_folder, f"R2_{cell_type}.csv")
    pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
    
    r2_df = pd.read_csv(r2_path, index_col=0)
    r2_df = r2_df[r2_df.index.isin(unique_samples)]
    pval_df = pd.read_csv(pval_path, index_col=0)
    pval_df = pval_df[pval_df.index.isin(unique_samples)]
    
    r2_df.columns = [f"{cell_type}_{col}" for col in r2_df.columns]
    pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
    
    combined_r2 = combined_r2.join(r2_df, how='left')
    combined_pval = combined_pval.join(pval_df, how='left')

    sample_counts_per_cell_type[cell_type] = r2_df.shape[0]
    for sample in r2_df.index:
        species = sample_to_species.get(sample, "Unknown")
        sample_counts_per_cell_type_per_species[species][cell_type] += 1

combined_pval.fillna(1, inplace=True)


# Provided cell types and colors
cell_colors = {
    "DCT_CNT_CD": "#3182bd",
    "DTL_ATL": "#fdd0a2",
    "EC": "seagreen",
    "IC": "orange",
    "Immune": "#c7e9c0",
    "Podo": "#000004",
    "Stromal": "limegreen",
    "PEC": "#fde725",
    "PT": "darkorchid",
    "TAL_MD": "lightcoral",
}

# Function to extract cell type from feature name
def extract_cell_type(feature_name):
    for cell_type in cell_colors.keys():
        if feature_name.startswith(cell_type):
            return cell_type
    return "Unknown"

# Sample data - replace with your actual data

top_20_features = combined_pval.loc[sample_of_interest].nsmallest(10)
neg_log_pvals = -np.log10(top_20_features)
r2_values = combined_r2.loc[sample_of_interest][top_20_features.index]

# Create a reversed red colormap for R2 values
cmap = plt.cm.Reds_r  # '_r' suffix to reverse the colormap

norm = mcolors.Normalize(vmin=0, vmax=1)  # Normalization fixed from 0 to 1

# Assuming 'significant_counts_df' is your DataFrame and it has a 'GeneSet' and 'NormalizedPercentage' column
scale_factor = 4
base_size = 100
#base_size = 20

# Map features to their normalized percentage
feature_to_percentage = dict(zip(significant_counts_df['GeneSet'], significant_counts_df['NormalizedPercentage']))

# Create the dot plot
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(r2_values[feature]))
    
    # Get the normalized percentage for the dot size
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(top_20_features.index, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)
ax.set_title(f'Top Features for Sample {sample_of_interest}', fontsize=14)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left')
ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()









# Create the dot plot
fig, ax = plt.subplots(figsize=(10, 5))
for i, (feature, neg_log_pval) in enumerate(neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    
    # Draw the bar for significance
    ax.plot([0, neg_log_pval], [i, i], color=bar_color, linewidth=3)


# Process each feature name through auto_split_label and set as y-tick labels
processed_labels = [auto_split_label(feat) for feat in top_20_features.index]
ax.set_yticks(range(len(top_20_features.index)))
ax.set_yticklabels(processed_labels, fontsize=12)

#ax.set_yticklabels(top_20_features.index, fontsize=20)
ax.set_xlabel('-log10(p-value)', fontsize=12)
ax.set_ylabel('Features', fontsize=12)

# Invert y-axis to have the most significant features at the top
ax.invert_yaxis()

plt.tight_layout()
plt.show()









# Define your desired order of cell types
desired_order = [
     
"PT", 
"TAL_MD", 
"DCT_CNT_CD", 
"IC",    
"EC",
"Stromal",      
"Immune",   
"DTL_ATL",  
"PEC",           
"Podo",    

    
]  # Replace with actual cell types

#top_20_features = combined_pval.loc[sample_of_interest].nsmallest(100)

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
plt.ylim(0, max_value)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

# Change the color of tick labels to red if the cell type is missing
for idx, label in enumerate(ax.get_xticklabels()):
    if desired_order[idx] not in categories:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


# Desired cell type for visualization
desired_cell_type = ct_plot1

# Assuming 'combined_pval' and 'combined_r2' are DataFrames with samples as rows and features as columns
# Filter for the specific sample of interest
sample_pvals = combined_pval.loc[sample_of_interest]
sample_r2 = combined_r2.loc[sample_of_interest]

# Filter the top features for the specific sample and cell type
#top_features_for_sample = sample_pvals.nsmallest(25)
filtered_features = sample_pvals[sample_pvals.index.map(extract_cell_type) == desired_cell_type].nsmallest(20)
filtered_neg_log_pvals = -np.log10(filtered_features)
filtered_r2_values = sample_r2[filtered_features.index]

# Now, update the dot plot code to use the filtered data
fig, ax = plt.subplots(figsize=(16, 5))
for i, (feature, neg_log_pval) in enumerate(filtered_neg_log_pvals.items()):
    cell_type = extract_cell_type(feature)
    bar_color = cell_colors.get(cell_type, 'grey')
    dot_color = cmap(norm(filtered_r2_values[feature]))
    
    # Get the normalized percentage for the dot size (if applicable)
    normalized_percentage = feature_to_percentage.get(feature, 0)
    dot_size = normalized_percentage * scale_factor + base_size

    ax.plot([0, neg_log_pval - (neg_log_pval / 10)], [i, i], color=bar_color, linewidth=3)
    ax.scatter(neg_log_pval, i, s=dot_size, color=dot_color, edgecolor='black', alpha=0.7, linewidth=0.5)

ax.set_yticks(range(len(filtered_features.index)))

prefix_to_remove = f"{desired_cell_type}_"
cleaned_labels = [label.replace(prefix_to_remove, '', 1) for label in filtered_features.index]

# Apply the auto-splitting to each label
cleaned_labels = [auto_split_label(label) for label in cleaned_labels]

ax.set_yticklabels(cleaned_labels, fontsize=10)
ax.set_xlabel('-log10(p-value)', fontsize=15)
#ax.set_ylabel('Features', fontsize=12)
#ax.set_title(f'Top Features for Sample {sample_of_interest} - Cell Type: {desired_cell_type}', fontsize=14)
ax.invert_yaxis()

# Create a colorbar for the R2 values
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, label='Scaled R2 Values', orientation='vertical')

# Legend for cell types
#cell_legend_elements = [plt.Line2D([0], [0], color=color, lw=4, label=cell_type) for cell_type, color in cell_colors.items()]
#legend1 = ax.legend(handles=cell_legend_elements, title='Cell Types', bbox_to_anchor=(1.5, 1), loc='upper left')

# Example normalized percentages for the dot size legend
example_percentages = [20, 40, 60, 80]
legend_dot_sizes = [p * scale_factor + base_size for p in example_percentages]

# Add legend for dot sizes
for p, size in zip(example_percentages, legend_dot_sizes):
    ax.scatter([], [], s=size, color='gray', edgecolor='black', alpha=0.7, label=f'{p}%')

legend2 = ax.legend(title='Frequency', bbox_to_anchor=(1.5, 0.2), loc='center left', fontsize='large')
#ax.add_artist(legend1)  # Add back the first legend

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import os

metadat = metadata

# Define the cell types and the base paths
cell_types = ["PT", "Podo", "TAL_MD", "DCT_CNT_CD", "EC", "Stromal", "IC", "Immune", "PEC", "DTL_ATL"]
#cell_types = ["Podo"]


# Function to read and process data for samples based on GFR threshold
def process_condition(metadat, unique_samples):


    
    
    combined_pval = pd.DataFrame(index=unique_samples)
    for cell_type in cell_types:
        pval_path = os.path.join(base_path, pval_folder, f"Pval_{cell_type}.csv")
        pval_df = pd.read_csv(pval_path, index_col=0)
        pval_df = pval_df[pval_df.index.isin(unique_samples)]
        pval_df.columns = [f"{cell_type}_{col}" for col in pval_df.columns]
        combined_pval = combined_pval.join(pval_df, how='left')
    return combined_pval

# Function to count significance
def count_significance(pval_df, threshold=0.01):
    sig_count = (pval_df < threshold).sum(axis=0, skipna=True)
    nonsig_count = (pval_df >= threshold).sum(axis=0, skipna=True)
    return sig_count, nonsig_count

# Read metadata



# Process each condition based on GFR threshold
#combined_pval_cond1 = process_condition(metadat, metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "m_humphreys_DKD")]["orig_ident"].tolist())  # Samples above the threshold
combined_pval_cond1 = process_condition(metadat, metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")]["orig_ident"].tolist())
combined_pval_cond2 = process_condition(metadat, metadata[(metadata["treated"] == "sGCstim") & (metadata["proj"] == "r_ZSF")]["orig_ident"].tolist())  # Samples below the threshold

# Count significance for each condition
sig_count_cond1, nonsig_count_cond1 = count_significance(combined_pval_cond1)
sig_count_cond2, nonsig_count_cond2 = count_significance(combined_pval_cond2)

# Creating the comparison DataFrame
comparison_df = pd.DataFrame(columns=combined_pval_cond1.columns, index=['cond1_sig', 'cond1_nonsig', 'cond2_sig', 'cond2_nonsig'])
comparison_df.loc['cond1_sig'] = sig_count_cond1
comparison_df.loc['cond1_nonsig'] = nonsig_count_cond1
comparison_df.loc['cond2_sig'] = sig_count_cond2
comparison_df.loc['cond2_nonsig'] = nonsig_count_cond2

# Display the DataFrame
#print(comparison_df)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact
from statsmodels.stats.multitest import multipletests

fisher_results = pd.DataFrame(columns=['Feature', 'Test_statistic', 'p_value', "odds_ratio", 'cond1_sig_count', 'cond2_sig_count'])

for feature in comparison_df.columns:
    # Creating the contingency table for each feature
    contingency_table = comparison_df[[feature]].values.reshape(2, 2)
    
    # Extract counts of significant samples for each condition for the current feature
    cond1_sig_count = comparison_df.loc['cond1_sig', feature]
    cond2_sig_count = comparison_df.loc['cond2_sig', feature]

    # Apply Fisher's Exact Test
    odds_ratio, p = fisher_exact(contingency_table)
    
    # Store the results
    result_row = pd.DataFrame([{
        'Feature': feature, 
        'Test_statistic': odds_ratio, 
        'p_value': p, 
        "odds_ratio": odds_ratio, 
        'cond1_sig_count': cond1_sig_count,
        'cond2_sig_count': cond2_sig_count
    }])
    
    fisher_results = pd.concat([fisher_results, result_row], ignore_index=True)

# Sort the results by p-value in ascending order
sorted_results = fisher_results.sort_values(by='p_value')

# Display the most significant gene sets
print(sorted_results.head())  # Adjust the number inside head() as needed

In [ ]:
# Extract the original p-values
p_values = sorted_results['p_value'].values

# Apply multiple test correction (using Benjamini-Hochberg method as an example)
correction_method = 'fdr_bh'  # You can change this to 'bonferroni', 'holm', etc., based on your requirement
rejected, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method=correction_method)

# Add the corrected p-values to your DataFrame
sorted_results['corrected_p_value'] = corrected_p_values
sorted_results['rejected_h0'] = rejected

# Now sorted_results has an additional column with corrected p-values
print(sorted_results.head())  # Adjust the number inside head() as needed


In [ ]:
sorted_results_subset = sorted_results[sorted_results["cond1_sig_count"] > 0]

sorted_results_subset.head(50)

In [ ]:

cell_type = "TAL_MD"

target_gene_set = "hsa04668 TNF signaling pathway - Homo sapiens (human)"

r2_base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"
pval_file_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Pval_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

#group0 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="Control_diseased")].orig_ident.unique().tolist()
group0 = metadata[(metadata['proj']=="r_ZSF") & (metadata['treated']=="Control_diseased")].orig_ident.unique().tolist()

#group1 = metadata[(metadata["proj"]=="m_humphreys_DKD") & (metadata["treated"]=="SLGT2i")].orig_ident.unique().tolist()
group1 = metadata[(metadata['proj']=="r_ZSF") & (metadata['treated']=="sGCact")].orig_ident.unique().tolist()

group2 = metadata[(metadata['proj']=="r_ZSF") & (metadata['treated']=="sGCstim")].orig_ident.unique().tolist()


samples_to_remove = []
samples_to_remove = []  # Replace with the names of samples you want to remove
  # Replace with the names of samples you want to remove

# Creating the list with samples to be removed
group0 = [sample for sample in group0 if sample not in samples_to_remove]

group1 = [sample for sample in group1 if sample not in samples_to_remove]

group2 = [sample for sample in group2 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.01]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.4, 0.6, 0.8]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group0, group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.9)  # Set the x-axis range
plt.xlabel(cell_type)
plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks(group_positions, ["DKD", "DKD + sGCact", "DKD + sGCstim"], rotation = 45)  # Set custom x-axis ticks
plt.yticks(fontsize=14)
#plt.legend()
plt.grid(False)
plt.show()

In [ ]:
cell_type = "TAL_MD"

target_gene_set = "hsa04668 TNF signaling pathway - Homo sapiens (human)"

r2_base_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"
pval_file_path = "/home/isilon/users/o_kloetzer/Atlas/Revision/SISKA/output_SISKA_crossspecies_healthy/"

# Load the R2 file
r2_file_path = f"{r2_base_path}/R2_KEGG_0/R2_{cell_type}.csv"
pval_file_path = f"{r2_base_path}/Pval_KEGG_0/Pval_{cell_type}.csv"
r2_df = pd.read_csv(r2_file_path, index_col=0)
pval_df = pd.read_csv(pval_file_path, index_col=0)

# Define the sample groups (time points)

#group1 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()
#group1 = metadata[(metadata["disease"] == "diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()
group1 = ["A3038", "B3039", "B3035", "J3036", "K3037"]

group2 = metadata[(metadata["treated"] == "SLGT2i") & (metadata["proj"] == "m_humphreys_DKD")].orig_ident.unique().tolist()
#group2 = metadata[(metadata["disease"] == "diseased") & (metadata["proj"] == "r_doca")].orig_ident.unique().tolist()


samples_to_remove = []
samples_to_remove = []  # Replace with the names of samples you want to remove


# Creating the list with samples to be removed
group1 = [sample for sample in group1 if sample not in samples_to_remove]

group2 = [sample for sample in group2 if sample not in samples_to_remove]

# Identify significant samples based on Pval < 0.05 for the specific gene set
significant_samples = pval_df.index[pval_df[target_gene_set] < 0.01]


# Assuming the rest of your setup code (loading data, etc.) is here

# Define x-coordinates for the groups, closer together
group_positions = [0.4, 0.6]  # Adjust these values as needed

# Create a figure for the plot
plt.figure(figsize=(2.5, 6))

# Initialize lists to store mean R2 values for each group
mean_r2_values = []

# Plot each individual sample at the new group positions
for i, group in enumerate([group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    mean_r2 = group_r2_values.mean()
    mean_r2_values.append(mean_r2)

    for sample in group:
        x_position = group_positions[i]  # x-coordinate for this group
        color = 'red' if sample in significant_samples else 'blue'
        plt.plot(x_position, group_r2_values.loc[sample], marker='o', markersize=5, color=color)

# Plot the mean R2 values
plt.plot(group_positions, mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

# Customize the plot
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1
plt.xlim(0.3, 0.7)  # Set the x-axis range
#plt.xlabel("Groups")
#plt.ylabel("R2 Value")
plt.title("rat")
plt.xticks(group_positions, ["ZSF", "DOCA"], fontsize=14)  # Set custom x-axis ticks
plt.yticks(fontsize=14)
#plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(False)
plt.show()

In [ ]:
# Create a DataFrame for plotting
plot_data = pd.DataFrame()
for i, group in enumerate([group1, group2]):
    group_r2_values = r2_df.loc[group, target_gene_set]
    group_data = pd.DataFrame({'R2 Value': group_r2_values, 'Group': i, 'Sample': group_r2_values.index})
    plot_data = pd.concat([plot_data, group_data])

# Add a column for color based on significance
plot_data['Color'] = plot_data['Sample'].apply(lambda x: 'red' if x in significant_samples else 'blue')

# Create a figure for the plot
plt.figure(figsize=(2, 6))

# Create a beeswarm plot

sns.swarmplot(x='Group', y='R2 Value', data=plot_data, hue='Color', palette={'red': 'red', 'blue': 'blue'})
plt.plot(mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2', zorder=10, markersize=12,
        markeredgewidth=2)

# Customize the plot
plt.ylim(0.0, 1)  # Set the y-axis range from 0 to 1
plt.xlabel(cell_type)
plt.ylabel("R2 Value")
plt.title(target_gene_set)
plt.xticks([0, 1], ["ZSF", "DOCA"])  # Set custom x-axis ticks
plt.legend().remove()  # Remove the legend if not needed
plt.grid(False)
plt.plot(mean_r2_values, marker='_', color='black', linestyle='', label='Mean R2')

plt.show()

In [ ]:
#top_20_features = combined_pval.loc[sample_of_interest].nsmallest(100)

sample_features = combined_pval.loc[sample_of_interest]

top_20_features = sample_features[sample_features < p_value_threshold]

# Assume 'combined_pval' is a DataFrame you have that contains the p-values

# Select the top 50 gene sets from the top features
top_gene_sets = top_20_features.index

# Group the top gene sets by cell type
grouped_data = pd.Series(top_gene_sets).apply(extract_cell_type).value_counts()

categories = list(grouped_data.index)

# Initialize ordered values with zero for all categories
ordered_values = [grouped_data.get(ct, 0) for ct in desired_order]

# Since we need to repeat the first value to close the circular graph
ordered_values += ordered_values[:1]

# Calculate the angle of each axis in the plot
N = len(desired_order)
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Initialize the spider plot
fig, ax = plt.subplots(figsize=(4, 4), subplot_kw=dict(polar=True))

# Draw one axis per variable and add labels
plt.xticks(angles[:-1], desired_order)

# Draw ylabels and set plot limits
ax.set_rlabel_position(0)
plt.yticks(color="grey", size=5)
plt.ylim(0, 700)


# Neutral color for shading
shade_color = 'lightgrey'

# Fill the entire area under the radar chart with a neutral color
ax.fill(angles, ordered_values, shade_color, alpha=0.9)

# Draw thin lines connecting the points
ax.plot(angles, ordered_values, color='grey', linewidth=1, linestyle='-', alpha=0.8)

# Plot each line segment in its respective cell type color
for idx in range(N):
    color = cell_colors.get(desired_order[idx], "grey")
    ax.plot([angles[idx], 0], [ordered_values[idx], 0], color=color, linewidth=2)

# Change the color of tick labels to red if the cell type is missing
for idx, label in enumerate(ax.get_xticklabels()):
    if desired_order[idx] not in categories:
        label.set_color('red')

# Add a title
plt.title('Cell Type Distribution in Top Gene Sets for Sample ' + sample_of_interest, size=11, y=1.1)

plt.show()


In [ ]:
# Sample groups
group1 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()
group2 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_doca")].orig_ident.unique().tolist()

color_list = ['green', 'lightgreen']

groups = {'ZSF': group1, 'DOCA': group2}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.01

    # Prepare DataFrame to hold percentages
    percentage_df = pd.DataFrame(columns=['Group', 'Percentage'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]

            # Count total non-NaN gene sets for the specific cell type
            total_non_nan = sample_features.notna().index.map(extract_cell_type).to_series().apply(lambda x: x == cell_type_of_interest).sum()

            if total_non_nan == 0:
                continue  # Skip if no non-NaN pathways for this cell type

            # Filter for significant features (below the p-value threshold)
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count significant gene sets for the specific cell type
            count_significant = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Calculate the percentage of significant gene sets relative to the total non-NaN gene sets
            percentage = (count_significant / total_non_nan) * 100

            # Append percentage to the DataFrame only if percentage is greater than zero
            if percentage > 0:
                new_row = pd.DataFrame({'Group': [group_name], 'Percentage': [percentage]})
                percentage_df = pd.concat([percentage_df, new_row], ignore_index=True)

    # Plotting the percentages using a box plot
    plt.figure(figsize=(5, 3))
    sns.boxplot(data=percentage_df, x='Group', y='Percentage', palette=color_list)
    sns.swarmplot(data=percentage_df, x='Group', y='Percentage', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Percentage of Significant Gene Sets')
    #plt.ylim(0, 100)
    plt.xlabel('Group')
    plt.show()


In [ ]:
# Sample groups
group1 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()

group2 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_doca")].orig_ident.unique().tolist()




color_list = ['green', 'lightgreen']

groups = {'ZSF': group1, 'DOCA': group2}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.01

    # Prepare DataFrame to hold counts
    counts_df = pd.DataFrame(columns=['Group', 'Count'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count gene sets for the specific cell type
            count = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Append count to the DataFrame only if count is greater than zero
            if count > 0:
                new_row = pd.DataFrame({'Group': [group_name], 'Count': [count]})
                counts_df = pd.concat([counts_df, new_row], ignore_index=True)

    # Plotting the counts using box plot
    plt.figure(figsize=(5, 3))
    sns.boxplot(data=counts_df, x='Group', y='Count', palette=color_list)
    sns.swarmplot(data=counts_df, x='Group', y='Count', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Number of Significant Gene Sets')
    plt.ylim(0)
    plt.xlabel('Group')
    plt.show()

In [ ]:


# Assuming metadata and combined_pval DataFrames are already defined
# Sample groups
group1 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()
group2 = metadata[(metadata["treated"] == "sGCact") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()
group3 = metadata[(metadata["treated"] == "sGCstim") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()

color_list = ['skyblue', 'lightgreen', 'salmon', "lightblue", "yellow"]

groups = {'Group 1': group1, 'Group 2': group2, 'Group 3': group3}

# Define the cell types of interest


for cell_type_of_interest in cell_types:
    p_value_threshold = 0.01

    # Prepare DataFrame to hold counts
    counts_df = pd.DataFrame(columns=['Group', 'Count'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count gene sets for the specific cell type
            count = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Append count to the DataFrame only if count is greater than zero
            if count > 0:
                new_row = pd.DataFrame({'Group': [group_name], 'Count': [count]})
                counts_df = pd.concat([counts_df, new_row], ignore_index=True)

    # Plotting the counts using box plot
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=counts_df, x='Group', y='Count', palette=color_list)
    sns.swarmplot(data=counts_df, x='Group', y='Count', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(f'Counts of Significant Gene Sets for {cell_type_of_interest}')
    plt.ylabel('Number of Significant Gene Sets')
    plt.xlabel('Group')
    plt.show()


In [ ]:
# Sample groups
group1 = metadata[(metadata["treated"] == "Control_diseased") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()

group2 = metadata[(metadata["treated"] == "sGCact") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()

group3 = metadata[(metadata["treated"] == "sGCstim") & (metadata["proj"] == "r_ZSF")].orig_ident.unique().tolist()

color_list = ['green', 'yellow', 'salmon']

groups = {'DKD': group1, 'DKD + sGCact': group2, 'DKD + sGCstim': group3}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.01

    # Prepare DataFrame to hold counts
    counts_df = pd.DataFrame(columns=['Group', 'Count'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count gene sets for the specific cell type
            count = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Append count to the DataFrame only if count is greater than zero
            if count > 0:
                new_row = pd.DataFrame({'Group': [group_name], 'Count': [count]})
                counts_df = pd.concat([counts_df, new_row], ignore_index=True)

    # Plotting the counts using box plot
    plt.figure(figsize=(5, 3))
    sns.boxplot(data=counts_df, x='Group', y='Count', palette=color_list)
    sns.swarmplot(data=counts_df, x='Group', y='Count', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Number of Significant Gene Sets')
    plt.ylim(0)
    plt.xlabel('Group')
    plt.show()

In [ ]:
# Sample groups
group1 = ["IRI4h1", "IRI4h2", "IRI4h3"]

group2 = ["IRI12h1b1",
"IRI12h1b2",
"IRI12h2",
"IRI12h3"]

group3 = ["IRI2d1b1",
"IRI2d1b2",
"IRI2d2b1",
"IRI2d2b2",
"IRI2d3"]

group4 = ["IRI14d1b1",
"IRI14d1b2",
"IRI14d2",
"IRI14d3"]

group5 = ["IRI6w1b1", "IRI6w1b2", "IRI6w2", "IRI6w3"]

color_list = ['skyblue', 'lightgreen', 'salmon', "lightblue", "yellow"]

groups = {'4h': group1, '12h': group2, '2d': group3, '14d': group4, '6W': group5}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.01

    # Prepare DataFrame to hold counts
    counts_df = pd.DataFrame(columns=['Group', 'Count'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count gene sets for the specific cell type
            count = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Append count to the DataFrame only if count is greater than zero
            if count > 0:
                new_row = pd.DataFrame({'Group': [group_name], 'Count': [count]})
                counts_df = pd.concat([counts_df, new_row], ignore_index=True)

    # Plotting the counts using box plot
    plt.figure(figsize=(10, 3))
    sns.boxplot(data=counts_df, x='Group', y='Count', palette=color_list)
    sns.swarmplot(data=counts_df, x='Group', y='Count', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Number of Significant Gene Sets')
    plt.ylim(0)
    plt.xlabel('Group')
    plt.show()

In [ ]:
# Sample groups
group1 = ["N3024", "A3026", "A3029", "B3025", "B3027", "C3028"]

group2 = ["A3038", "B3039", "B3035", "J3036", "K3037"]




color_list = ['yellow', 'orange']

groups = {'db/db': group1, 'db/db+AAV': group2}

for cell_type_of_interest in cell_types:
    p_value_threshold = 0.01

    # Prepare DataFrame to hold counts
    counts_df = pd.DataFrame(columns=['Group', 'Count'])

    # Loop through each group and each sample within the group
    for group_name, samples in groups.items():
        for sample in samples:
            # Filter for cell type of interest and significant p-values
            sample_features = combined_pval.loc[sample]
            significant_features = sample_features[sample_features < p_value_threshold]
            top_gene_sets = significant_features.index

            # Count gene sets for the specific cell type
            count = pd.Series(top_gene_sets).apply(lambda x: x.startswith(cell_type_of_interest)).sum()

            # Append count to the DataFrame only if count is greater than zero
            if count > 0:
                new_row = pd.DataFrame({'Group': [group_name], 'Count': [count]})
                counts_df = pd.concat([counts_df, new_row], ignore_index=True)

    # Plotting the counts using box plot
    plt.figure(figsize=(5, 3))
    sns.boxplot(data=counts_df, x='Group', y='Count', palette=color_list)
    sns.swarmplot(data=counts_df, x='Group', y='Count', color='black', alpha=0.5)  # Add swarmplot for individual points
    plt.title(cell_type_of_interest)
    plt.ylabel('Number of Significant Gene Sets')
    plt.ylim(0)
    plt.xlabel('Group')
    plt.show()